In [ ]:
import os
import pandas as pd
import datetime
import re
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    return chat_files

def parse_chat_file(file_path, expected_date_minus_one):
    chat_data = []
    last_person_time = None
    last_sender = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            logging.debug(f"Reading message: {line.strip()}")
            message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
            system_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*)', line)
            if message_match:
                date_time_str, sender, message = message_match.groups()
            elif system_match:
                date_time_str, info = system_match.groups()
                sender = None
            else:
                continue

            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')
            if date_time.date() != expected_date_minus_one:
                continue

            is_person = sender is not None and re.match(r'^[+\d\s-]+$', sender) is None
            delay = False
            if is_person:
                if last_person_time and sender != last_sender and (date_time - last_person_time).total_seconds() > 900:
                    delay = True
                if not delay:
                    last_person_time = date_time
                    last_sender = sender

            chat_data.append((date_time, sender, is_person, delay))

    return chat_data, extract_group_name(file_path)



def create_template_dataframe():
    times = [datetime.datetime(2000, 1, 1, 0, 0) + datetime.timedelta(minutes=1 * i) for i in range(1440)]
    intervals = [time.strftime('%I:%M %p') for time in times]
    df = pd.DataFrame(index=intervals)
    return df

def populate_dataframe(df, parsed_data, group_name):
    # Ensure all three columns for each chat
    for suffix in ['', '_others', '_delay']:
        column_name = f"{group_name}{suffix}"
        if column_name not in df.columns:
            df[column_name] = 0

    # Populate the DataFrame with parsed data
    for date_time, sender, is_person, delay in parsed_data:
        interval_index = min((date_time.hour * 60 + date_time.minute) // 1, 1439)
        interval = df.index[interval_index]

        # Update person, others, and delay columns
        df.at[interval, group_name] = 1 if is_person else df.at[interval, group_name]
        df.at[interval, f"{group_name}_others"] = 0 if is_person else 1
        df.at[interval, f"{group_name}_delay"] = 1 if delay else df.at[interval, f"{group_name}_delay"]

    # Calculate active chats for each interval
    if 'active_chat' not in df.columns:
        df['active_chat'] = 0
    for interval in df.index:
        active_chats = 0
        for col in df.columns:
            if '_others' in col or group_name in col:  # Check only person and others columns
                if df.at[interval, col] == 1:
                    active_chats += 1
                    break  # Break as we only need at least one active indicator per chat
        df.at[interval, 'active_chat'] = active_chats

    return df



def extract_group_name(file_path):
    group_name = os.path.basename(file_path).replace('WhatsApp Chat with ', '').split('.')[0]
    group_name = re.sub(r'\(\d+\)$', '', group_name)  # Remove any numbers in parentheses at the end
    return group_name

date_directory = "C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231201T052455Z-001"
chat_files = list_chat_files(date_directory)
dataframes = {}

for file in chat_files:
    parts = file.split(os.sep)
    date_folder, person = parts[-4], parts[-2]

    try:
        folder_date = pd.to_datetime(date_folder, format='%Y-%m-%d').date()
    except ValueError:
        continue

    expected_date_minus_one = folder_date - datetime.timedelta(days=1)
    key = f"{folder_date.strftime('%Y-%m-%d')}_{person}"

    if key not in dataframes:
        dataframes[key] = create_template_dataframe()
    parsed_data, group_name = parse_chat_file(file, expected_date_minus_one)
    dataframes[key] = populate_dataframe(dataframes[key], parsed_data, group_name)


In [ ]:
# Loop through the dictionary of DataFrames
for key, df in dataframes.items():
    print(f"DataFrame for {key}:\n{df.head()}")


In [ ]:
# Directory to save CSV files
csv_save_directory = "C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat CSVs"
os.makedirs(csv_save_directory, exist_ok=True)

# Saving each DataFrame as a CSV
for key, df in dataframes.items():
    csv_file_path = os.path.join(csv_save_directory, f"{key}.csv")
    df.to_csv(csv_file_path)

In [1]:
import os
import pandas as pd
import datetime
import re
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                # Only proceed if the team folder is 'KAM'
                if team_folder != "KAM":
                    continue
                
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    return chat_files


def parse_chat_file(file_path, expected_date_minus_one):
    chat_data = []
    last_person_time = None
    last_sender = None
    delay_count = 0

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            
            message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
            system_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*)', line)
            if message_match:
                date_time_str, sender, message = message_match.groups()
            elif system_match:
                date_time_str, info = system_match.groups()
                sender = None
            else:
                continue

            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')
            if date_time.date() != expected_date_minus_one:
                continue

            is_person = sender is not None and re.match(r'^[+\d\s-]+$', sender) is None
            delay = False
            if is_person:
                if last_person_time and sender != last_sender and (date_time - last_person_time).total_seconds() > 900:
                    delay = True
                    delay_count += 1
                if not delay:
                    last_person_time = date_time
                    last_sender = sender

            chat_data.append((date_time, sender, is_person, delay))
    logging.debug(f"File parsed: {file_path}. Delays detected: {delay_count}")
    return chat_data, extract_group_name(file_path)



def create_template_dataframe():
    times = [datetime.datetime(2000, 1, 1, 0, 0) + datetime.timedelta(minutes=1 * i) for i in range(1440)]
    intervals = [time.strftime('%I:%M %p') for time in times]
    df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
    return df


def populate_dataframe(df, parsed_data, group_name):
    # Create new columns as separate DataFrames
    new_columns = {
        f"{group_name}": pd.Series(0, index=df.index),
        f"{group_name}_others": pd.Series(0, index=df.index),
        f"{group_name}_delay": pd.Series(0, index=df.index)
    }

    # Populate the new columns with parsed data
    for date_time, sender, is_person, delay in parsed_data:
        interval_index = min((date_time.hour * 60 + date_time.minute) // 1, 1439)
        interval = df.index[interval_index]

        # Update person, others, and delay columns
        new_columns[f"{group_name}"].at[interval] = 1 if is_person else new_columns[f"{group_name}"].at[interval]
        new_columns[f"{group_name}_others"].at[interval] = 0 if is_person else 1
        new_columns[f"{group_name}_delay"].at[interval] = 1 if delay else new_columns[f"{group_name}_delay"].at[interval]

    # Concatenate the new columns to the original DataFrame
    df = pd.concat([df, pd.DataFrame(new_columns)], axis=1)

    # Vectorized approach to calculate active chats
    if 'active_chat' not in df.columns:
        df['active_chat'] = 0

    # Filter columns for the current group
    relevant_columns = [col for col in df.columns if col.startswith(group_name) and ('_others' in col or '_person' in col)]
    df['active_chat'] = df[relevant_columns].any(axis=1).astype(int)

    return df


def extract_group_name(file_path):
    group_name = os.path.basename(file_path).replace('WhatsApp Chat with ', '').split('.')[0]
    group_name = re.sub(r'\(\d+\)$', '', group_name)  # Remove any numbers in parentheses at the end
    return group_name

date_directory = "C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231201T052455Z-001"
chat_files = list_chat_files(date_directory)
dataframes = {}

for file in chat_files:
    parts = file.split(os.sep)
    date_folder, person = parts[-4], parts[-2]

    try:
        folder_date = pd.to_datetime(date_folder, format='%Y-%m-%d').date()
    except ValueError:
        continue

    expected_date_minus_one = folder_date - datetime.timedelta(days=1)
    key = f"{folder_date.strftime('%Y-%m-%d')}_{person}"

    if key not in dataframes:
        dataframes[key] = create_template_dataframe()
    parsed_data, group_name = parse_chat_file(file, expected_date_minus_one)
    dataframes[key] = populate_dataframe(dataframes[key], parsed_data, group_name)


C:\Users\mauriceyeng\AppData\Local\Temp\ipykernel_17072\2244814449.py:72: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
2023-12-01 17:16:12,163 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Ashi_Edoofa\WhatsApp Chat with Aaron 2K23OCT1666R(1).txt. Delays detected: 0
2023-12-01 17:16:12,212 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Ashi_Edoofa\WhatsApp Chat with Aaron 2K23OCT1666R.txt. Delays detected: 0
2023-12-01 17:16:12,246 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12

2023-12-01 17:16:13,517 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Ashi_Edoofa\WhatsApp Chat with Dalitso 2K23OCT0589.txt. Delays detected: 0
2023-12-01 17:16:13,541 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Ashi_Edoofa\WhatsApp Chat with Danai 2K23SEP1134R.txt. Delays detected: 0
2023-12-01 17:16:13,594 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Ashi_Edoofa\WhatsApp Chat with Denzel 2K23OCT1175.txt. Delays detected: 0
2023-12-01 17:16:13,628 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Ashi_Edoofa\WhatsApp Chat with Fanwell 2K22NOV1427.txt. Delays detected: 0
2023-12-01 17:16:13,716 - DEBUG - File parsed: C:\

2023-12-01 17:16:14,955 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Ashi_Edoofa\WhatsApp Chat with Shalom 2K21MAY0347.txt. Delays detected: 0
2023-12-01 17:16:15,004 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Ashi_Edoofa\WhatsApp Chat with Shantelle 2K23SEP2868R.txt. Delays detected: 0
2023-12-01 17:16:15,046 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Ashi_Edoofa\WhatsApp Chat with Sharmaine 2K23OCT0695.txt. Delays detected: 0
2023-12-01 17:16:15,088 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Ashi_Edoofa\WhatsApp Chat with Shumba 2K23SEP1992.txt. Delays detected: 0
2023-12-01 17:16:15,115 - DEBUG - File parsed

2023-12-01 17:16:16,449 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Kirti Edoofa\WhatsApp Chat with Gombe 2K23OCT0971R.txt. Delays detected: 0
2023-12-01 17:16:16,492 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Kirti Edoofa\WhatsApp Chat with Isaac 2K23OCT1366.txt. Delays detected: 0
2023-12-01 17:16:16,546 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Kirti Edoofa\WhatsApp Chat with Joy 2K23OCT2614R.txt. Delays detected: 0
2023-12-01 17:16:16,610 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Kirti Edoofa\WhatsApp Chat with Kudzaishe 2K23AUG1288.txt. Delays detected: 0
2023-12-01 17:16:16,658 - DEBUG - File parsed: C

2023-12-01 17:16:18,018 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Milan_Edoofa\WhatsApp Chat with David 2K22AUG2395.txt. Delays detected: 0
2023-12-01 17:16:18,060 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Milan_Edoofa\WhatsApp Chat with David 2K23MAR0605.txt. Delays detected: 0
2023-12-01 17:16:18,101 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Milan_Edoofa\WhatsApp Chat with Diana 2K23MAR2362.txt. Delays detected: 0
2023-12-01 17:16:18,157 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Milan_Edoofa\WhatsApp Chat with Emily 2K23SEP0768.txt. Delays detected: 0
2023-12-01 17:16:18,220 - DEBUG - File parsed: C:\Us

2023-12-01 17:16:19,567 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Milan_Edoofa\WhatsApp Chat with Sibonginkosi 2K23JUN1094.txt. Delays detected: 0
2023-12-01 17:16:19,609 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Milan_Edoofa\WhatsApp Chat with Tadiwanashe 2K22JUN1652.txt. Delays detected: 0
2023-12-01 17:16:19,643 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Milan_Edoofa\WhatsApp Chat with Tadiwanashe 2K23MAR2892R.txt. Delays detected: 0
2023-12-01 17:16:19,699 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Milan_Edoofa\WhatsApp Chat with Takunda 2K23AUG0788.txt. Delays detected: 0
2023-12-01 17:16:19,734 - DEBU

2023-12-01 17:16:20,838 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Shivjeet Edoofa\WhatsApp Chat with Malcom 2K23NOV2028.txt. Delays detected: 0
2023-12-01 17:16:20,922 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Shivjeet Edoofa\WhatsApp Chat with Master 2K23AUG1589.txt. Delays detected: 0
2023-12-01 17:16:20,970 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Shivjeet Edoofa\WhatsApp Chat with Mbuso 2K23SEP2424.txt. Delays detected: 0
2023-12-01 17:16:21,004 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Shivjeet Edoofa\WhatsApp Chat with Moreblessing 2K23APR1124.txt. Delays detected: 0
2023-12-01 17:16:21,039 - DEBUG

In [2]:
# Directory to save CSV files
csv_save_directory = "C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat CSVs"
os.makedirs(csv_save_directory, exist_ok=True)

# Saving each DataFrame as a CSV
for key, df in dataframes.items():
    csv_file_path = os.path.join(csv_save_directory, f"{key}.csv")
    df.to_csv(csv_file_path)